# Filter the `Hellisotherpeople/DebateSum` dataset for hateful content / extremist content

The `Hellisotherpeople/DebateSum` has a lot of radical & extremist content, like pro-marxist/communist advocacy and racist-activism

In [1]:
%pip install pystemmer datasets xgboost transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 27.6 MB/s eta 0:00:00
  Created wheel for pystemmer: filename=PyStemmer-2.2.0.1-cp310-cp310-linux_x86_64.whl size=579706 sha256=e27e8ff40a63e1f4c75bd6b28c1d0f93dbe76b9cffc42c707db3f348813f1faf
  Stored in directory: /root/.cache/pip/wheels/45/7d/2c/a7ebb8319

In [2]:
import torch
from torch.cuda import is_available
assert is_available()
#foo =load_dataset('Hellisotherpeople/DebateSum',split='train',streaming=True)
import pickle
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import string
import Stemmer
stemmer = Stemmer.Stemmer("english")
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from scipy.stats import scoreatpercentile
import warnings
warnings.filterwarnings("ignore")

In [3]:
data = load_dataset("Hellisotherpeople/DebateSum",split='train',streaming=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [4]:


def is_extremist(d):
    x = d['Full-Document']
    if 'LatCrit' in x or 'critical consciousness' in x.lower():
       return True
    if (
        'imperialist' in x or 'imperialism' in x.lower() or 'anti-colonial' in x.lower()
    ) and (
        'anti-capitalis' in x or 'discourse' in x or 'antiracist' in x or 'postmodern' in x or 'social justice' in x.lower()
    ):
       return True
    if 'Karl Marx' in x or 'poststructuralist' in x or 'Marist' in x or 'Marxism' in x or 'Trotsky' in x or 'Karl Marx' in x:
       return True
    if 'queer' in x or 'LGBTQ' in x or 'LGBT' in x or 'heteronormative' in x.lower() or 'hetero-normative' in x.lower():
       return True
    if 'misogyny' in x.lower() or 'Patriarchy' in x or 'ecofeminist' in x or 'patriarchies' in x or 'feminist' in x.lower() or 'sexism' in x:
       return True
    if 'straight white male' in x or 'white, heterosexual, and European men' in x.lower() or 'Western man' in x or 'homophobic' in x or 'xenophobic' in x:
       return True
    if (
        'settler' in x or 'indigenous movement' in x.lower() or 'indigenous activ' in x.lower() or ' land back' in x or 'decolonize' in x.lower() or 'BIPOC' in x.lower() or 'BLM' in x or
        'white suprem' in x.lower() or 'eurocentric' in x.lower() or 'rapper' in x or 'bitch' in x or 'hip-hop' in x or 'super predators' in x or 'crack babies' in x
    ):
       return True
    if 'trans-movement' in x or 'trans-identity' in x:
       return True
    if 'critical theory' in x or 'praxis' in x:
       return True
    return False

# get streaming-object for extremist text
#l = load_dataset("Hellisotherpeople/DebateSum", split='train',streaming=True).filter(is_extremist)

# get candidates for extremist text
#ltexts = [e for e in l]

ltexts = [e for e in data.filter(is_extremist)]


Filter:   0%|          | 0/240566 [00:00<?, ? examples/s]

In [7]:
print(f"{len(ltexts)} possible extremist texts")

16074 possible extremist texts


In [8]:
ltexts[-1000]

{'Unnamed: 0': 226928,
 'Full-Document': 'Judicial politics are contradictory like that. Any strategy for responding to the Gorsuch nomination and Trump’s judicial agenda more broadly must begin with an appreciation of these paradoxes, and a clear-eyed view of the Supreme Court’s conservative constitutional role. It is not a matter of opposing Gorsuch because of his originalism — still less of defending the judiciary from capture by conservative jurisprudence. It is a matter of resisting the judiciary itself as a reactionary institution. It is about focusing on the Court, not on Trump’s pick. A movement of mass opposition and disruption must target institutions rather than individuals, power rather than personalities. A filibuster by Senate Democrats would certainly be a welcome (and newfound) display of parliamentary intransigence. But it would be a mistake to stake all our hopes on the success of a campaign of protest aimed at holding the Democrats to account. There are few reasons t

In [9]:

corpus = [d['Full-Document'] for d in ltexts] + [d['Extract'] for d in ltexts]
corpus_names = [d['OriginalDebateFileName'] for d in ltexts] + [d['OriginalDebateFileName'] for d in ltexts]

# Define a function to preprocess a single text
def preprocess_text(text):
    """Preprocessing pipeline to convert to a (small) tfidf vector focused on extremist texts"""
    # Convert to lowercase
    text = text.lower()
    # Remove URLs using a regular expression
    text = re.sub(r'http\S+', '', text)
    # Remove special characters and punctuation
    text = ''.join([char if (char not in ['-',"'", '"', '‘', '’', '/']) else " " for char in text])
    text = ''.join([char for char in text if char not in string.punctuation])
    # Split the text into words
    words = text.split()
    # Remove stopwords (you can replace this list with your own)
    stopwords = set([
        "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself","it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now", "d", "ll", "m", "o", "re", "ve", "y", "ain", "aren", "couldn", "didn", "doesn", "hadn", "hasn", "haven", "isn", "ma", "mightn", "mustn", "needn", "shan", "shouldn", "wasn", "weren", "won", "wouldn","also"
    ])
    #words = [word for word in words if word not in stopwords]
    words = [stemmer.stemWord(word) for word in words if word not in stopwords]
    # Join the words back into a single text
    preprocessed_text = ' '.join(words)
    return preprocessed_text

# Apply preprocessing to each text in list_of_texts
preprocessed_texts = [preprocess_text(text) for text in corpus]
# Create a TfidfVectorizer with your preprocessed texts
#vectorizer = TfidfVectorizer(max_features=2000)
#vectorizer_1000 = TfidfVectorizer(max_features=1000)
vectorizer = TfidfVectorizer(max_features=500)
# Fit the vectorizer on the preprocessed texts and transform them into TF-IDF vectors
tfidf_matrix = vectorizer.fit_transform(preprocessed_texts)
# Calculate the global centroid of TF-IDF vectors
centroid = np.mean(tfidf_matrix, axis=0)
# Calculate cosine distances from the centroid to all vectors in the corpus
distances = cosine_distances(np.asarray(centroid), tfidf_matrix)



In [10]:
# Specify the quantiles you want to compute
quantiles_to_compute = [0.05, 0.2, 0.5, 0.6, 0.75, 0.8, 0.9, 0.95, 0.99]
# Calculate the specified quantiles of distances
quantiles = {q:scoreatpercentile(distances, q * 100) for q in quantiles_to_compute}
# filterout those texts that are probably not extremist
quantile_threshold = 0.6

## the positive set for classifying

In [11]:
ltests2_docs = [l for l,d in zip(ltexts, distances.reshape(-1)) if d<quantiles[quantile_threshold]]
ltests2 = [s for s,d in zip(corpus, distances.reshape(-1)) if d<quantiles[quantile_threshold]]
# THIS IS THE POSITIVE SET
print(len(ltests2))

19289


In [12]:
# the negative set
def not_extremist(d):
    """silver data set of (not) extremist text to act as an pseudo-negative"""
    if len(str(d['Full-Document']).split(' ')) < 500:
        return False
    return not is_extremist(d)

# get non-extremist content
#notl = load_dataset("Hellisotherpeople/DebateSum", split='train',streaming=True).filter(not_extremist).take(10000) # a subsample that are not extremist texts
notltexts = [e for e in data.filter(not_extremist)][:10000]
#corpus = [d['Full-Document'] for d in notltexts]

Filter:   0%|          | 0/240566 [00:00<?, ? examples/s]

In [16]:
print(f"size of pseudo-negative set:{len(notltexts)}")

size of pseudo-negative set:10000


### Make semantic features (using a small embedding NN like miniLM or e5)

In [15]:

import torch.nn.functional as F
from torch import Tensor
from transformers import AutoTokenizer, AutoModel
import torch

tokenizer = AutoTokenizer.from_pretrained('intfloat/e5-small-v2')
model = AutoModel.from_pretrained('intfloat/e5-small-v2')
model = model.to('cuda:0')

def average_pool(
    last_hidden_states: Tensor,
    attention_mask: Tensor
) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

In [17]:



def batch_encode_text(texts, batch_size=64):
    """Batch encoding of semantic features; sentence-vectors (no backpropagation)"""
    # Initialize the tokenizer and model
    # Initialize lists to store the embeddings
    embeddings_all = []
    # Process the texts in batches
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        # Tokenize the batch
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        # Encode the batch
        for k,v in inputs.items():
            inputs[k] = v.to('cuda:0')
        with torch.no_grad():
            outputs = model(**inputs)

        # Extract the embeddings (e.g., the last hidden states)
        embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])
        embeddings = embeddings.cpu().detach().numpy()
        # Add the batch of embeddings to the list
        embeddings_all.append(embeddings)

    # Concatenate the embeddings from all batches
    concatenated_embeddings = np.concatenate(embeddings_all,axis=0)
    return concatenated_embeddings

# Tokenize the input texts
#batch_dict = tokenizer(["it mystifies a world that cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather than less radical, using whatever re- sources are available to us as teachers and practitioners to expose the myths of liberal as well as conservative world-views. Speaking to law teacher"], max_length=512, padding=True, truncation=True, return_tensors='pt')
#batch_encode_text(["it mystifies a world that cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather than less radical, using whatever re- sources are available to us as teachers and practitioners to expose the myths of liberal as well as conservative world-views. Speaking to law teacher", "cal rather than less radical, using whatever re- sources are available to us as teachers and practitioners to"," cannot actually satisfy people\'s real human needs. But to do this we must be more radical rather tha"], batch_size=2)
#outputs = model(**batch_dict)
#embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# convert all the texts to vectors
def pipeline_for_classification(texts):
    # tfidf
    tfidf_vectors = vectorizer.fit_transform([
        preprocess_text(text) for text in texts
    ]).toarray()
    # then minilm
    nn_outputs = batch_encode_text(texts, batch_size=64)
    return np.concatenate((tfidf_vectors, nn_outputs), axis=1)


In [18]:
# run silver-data corpus through NN feature-pipeline
# WARNING: this takes a LONG time

features_notl = pipeline_for_classification([str(d['Full-Document']) for d in notltexts])
# make classification features for the leftist texts
features_l = pipeline_for_classification([str(s) for s in ltests2])
# make targets
targets = [0.0]*features_notl.shape[0] + [1.0]*features_l.shape[0]


In [19]:
import pickle

In [20]:
# save corpus and session for later
with open('extremist-text-classification.pkl','wb') as pcon:
    pickle.dump(features_notl, pcon)
    pickle.dump(features_l, pcon)
    pickle.dump(targets, pcon)
    pickle.dump(notltexts, pcon)
    pickle.dump(ltests2, pcon)
    pickle.dump(tokenizer, pcon)

In [ ]:
with open('extremist-text-classification.pkl','rb') as pcon:
    features_notl = pickle.load(pcon)
    features_l = pickle.load(pcon)
    targets = pickle.load(pcon)
    notltexts = pickle.load(pcon)
    ltests2 = pickle.load(pcon)
    tokenizer = pickle.load(pcon)


In [21]:
# XGB classification

import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Assuming you have your features in 'features' and your binary targets in 'targets'

# Split the data into training and holdout sets
X_train, X_holdout, y_train, y_holdout = train_test_split(
    np.concatenate((features_notl, features_l),axis=0),
    targets, test_size=0.1, random_state=42
)

# Convert the data to DMatrix format (required by XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dholdout = xgb.DMatrix(X_holdout, label=y_holdout)

# Define XGBoost parameters
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',  # You can choose another metric like 'auc' if desired
    'max_depth': 5,
    'eta': 0.27,
    'gamma': 0.1,
    'min_child_weight': 12.0,
    'subsample': 0.3,
    'colsample_bytree': 0.5,
    'scale_pos_weight': 0.75,
    'seed': 42
}

# Specify the number of boosting rounds and early stopping criteria
num_rounds = 500  # You can adjust this number
early_stopping_rounds = 20

# Train the XGBoost model with early stopping
watchlist = [(dtrain, 'train'), (dholdout, 'eval')]
xgbmod = xgb.train(params, dtrain, num_rounds, evals=watchlist,
                  early_stopping_rounds=early_stopping_rounds, verbose_eval=True)

# Make predictions on the holdout set
y_pred = xgbmod.predict(dholdout, ntree_limit=xgbmod.best_iteration)

# Convert predicted probabilities to binary labels
y_pred_binary = [1 if p >= 0.5 else 0 for p in y_pred]

# Evaluate the model on the holdout set
accuracy = accuracy_score(y_holdout, y_pred_binary)
print(f"Accuracy on Holdout Set: {accuracy}")


[0]	train-logloss:0.52268	eval-logloss:0.52273
[1]	train-logloss:0.41302	eval-logloss:0.41337
[2]	train-logloss:0.32866	eval-logloss:0.32882
[3]	train-logloss:0.27085	eval-logloss:0.27121
[4]	train-logloss:0.22522	eval-logloss:0.22713
[5]	train-logloss:0.18966	eval-logloss:0.19187
[6]	train-logloss:0.16179	eval-logloss:0.16465
[7]	train-logloss:0.13860	eval-logloss:0.14083
[8]	train-logloss:0.11957	eval-logloss:0.12163
[9]	train-logloss:0.10313	eval-logloss:0.10538
[10]	train-logloss:0.09008	eval-logloss:0.09221
[11]	train-logloss:0.07885	eval-logloss:0.08080
[12]	train-logloss:0.07049	eval-logloss:0.07240
[13]	train-logloss:0.06273	eval-logloss:0.06495
[14]	train-logloss:0.05606	eval-logloss:0.05844
[15]	train-logloss:0.05043	eval-logloss:0.05285
[16]	train-logloss:0.04581	eval-logloss:0.04810
[17]	train-logloss:0.04161	eval-logloss:0.04354
[18]	train-logloss:0.03823	eval-logloss:0.04000
[19]	train-logloss:0.03480	eval-logloss:0.03634
[20]	train-logloss:0.03204	eval-logloss:0.03349
[2

In [22]:
def pipeline_for_inference(texts):
    # tfidf
    tfidf_vectors = vectorizer.fit_transform([
        preprocess_text(text) for text in texts
    ]).toarray()
    # then minilm
    nn_outputs = batch_encode_text(texts, batch_size=32)
    # features for xgboosr
    features_inference =np.concatenate((tfidf_vectors, nn_outputs), axis=1)
    X_inference = xgb.DMatrix(features_inference)
    return xgbmod.predict(X_inference)

def pipelines_for_inference_batched(inputs):
    pred = pipeline_for_inference(inputs['Full-Document'])
    return {
        'pred':pred.tolist(),
    }

# stream and make predictions on data





In [ ]:
## Try to predict on the entire dataset

## Try to predict on the entire dataset (inference)

This MAY be possible using GPUs in a short time... otherwise I'll have to classify on a select few

In [23]:
do_streaming = False

if do_streaming:
    # WARNING: this is too much
    # get the full dataset and those which are very high in pred
    predset_streaming = load_dataset(
        'Hellisotherpeople/DebateSum',split='train',streaming=True
    ).filter(
        lambda x : (len(x['Full-Document'].split(' '))>400)
    ).map(
        #pipelines_for_inference_and_insert_into_example, batched = True, batch_size=32
        pipelines_for_inference_batched, batched = True, batch_size=32*8
    ).remove_columns(
        ['#CharsAbstract', '#CharsDocument', '#CharsExtract', '#WordsAbstract', '#WordsDocument', '#WordsExtract', 'AbsCompressionRatio', 'Abstract', 'Citation', 'DebateCamp', 'ExtCompressionRatio', 'Extract', 'Tag', 'Unnamed: 0', 'Year']
    ).take(10)

    # takes too long: instead, I should first filter done by keywords: (white suprem
    predset = [e for e in predset_streaming]

else:

    # classify the texts
    predset = data.filter(
        lambda x : (len(x['Full-Document'].split(' '))>400)
    ).map(
        #pipelines_for_inference_and_insert_into_example, batched = True, batch_size=32
        pipelines_for_inference_batched, batched = True, batch_size=32*16
    ).remove_columns(
        ['#CharsAbstract', '#CharsDocument', '#CharsExtract', '#WordsAbstract', '#WordsDocument', '#WordsExtract', 'AbsCompressionRatio', 'Abstract', 'Citation', 'DebateCamp', 'ExtCompressionRatio', 'Extract', 'Tag', 'Unnamed: 0', 'Year']
    )

Filter:   0%|          | 0/240566 [00:00<?, ? examples/s]

Parameter 'function'=<function pipelines_for_inference_batched at 0x792769c820e0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/105105 [00:00<?, ? examples/s]

#### Done inference/prediction

This is actually pretty reasonible, time-wise, for the entire dataset.

In [26]:
predset[-1]

{'Full-Document': 'Converging technologies also pose fundamental human security challenges. As Francis Fukuyama once argued in Our Posthuman Future: Consequences of the Biotechnology Revolution, converging technologies could inequitably transform the world we live in and, in the process, undermine the very foundations that underpin liberal democracies. Whether or not such a future unfolds, it is clear that their application raises serious ethical and moral issues that are proving divisive for allies and enemies alike (eg: the debates over armed drones and cyber espionage). If NATO member states want to sustain “the traditional transatlantic compact [European political support in return for US military guarantees]”, they must change the way NATO approaches cooperative security around emerging technologies. And, they need to do it now. First and foremost, NATO needs to substantially increase its investment in its Emerging Security Challenges Division (ESCD). This division should be provi

In [30]:
# calculate the percentiles of the predictions
q_preds = np.quantile([p['pred'] for p in predset], [i/100 for i in range(1,100)])
q_preds = {q:v for q,v in zip([i/100 for i in range(1,100)], q_preds)}
print(q_preds)


{0.01: 0.0007588447304442524, 0.02: 0.001307484097778797, 0.03: 0.0019421369349583982, 0.04: 0.0025906706508249046, 0.05: 0.0033049640245735646, 0.06: 0.004075787290930747, 0.07: 0.004905904922634364, 0.08: 0.005795443560928106, 0.09: 0.006724981386214493, 0.1: 0.007721059489995242, 0.11: 0.008882404267787934, 0.12: 0.010038076750934124, 0.13: 0.01137153834104538, 0.14: 0.01278867892920971, 0.15: 0.014240797422826284, 0.16: 0.015917663499712943, 0.17: 0.017649030908942223, 0.18: 0.01943587385118007, 0.19: 0.021331265196204187, 0.2: 0.023453339189291003, 0.21: 0.025736050605773927, 0.22: 0.028136154115200047, 0.23: 0.03063242472708226, 0.24: 0.0333886045217514, 0.25: 0.036214739084243774, 0.26: 0.03936804011464119, 0.27: 0.042488856017589574, 0.28: 0.04582953661680224, 0.29: 0.049509342014789584, 0.3: 0.053396073728799814, 0.31: 0.05745161607861519, 0.32: 0.06184772193431854, 0.33: 0.0666947540640831, 0.34: 0.07166790306568147, 0.35: 0.076985427737236, 0.36: 0.082470656633377, 0.37: 0.0

In [31]:
q_preds

{0.01: 0.0007588447304442524,
 0.02: 0.001307484097778797,
 0.03: 0.0019421369349583982,
 0.04: 0.0025906706508249046,
 0.05: 0.0033049640245735646,
 0.06: 0.004075787290930747,
 0.07: 0.004905904922634364,
 0.08: 0.005795443560928106,
 0.09: 0.006724981386214493,
 0.1: 0.007721059489995242,
 0.11: 0.008882404267787934,
 0.12: 0.010038076750934124,
 0.13: 0.01137153834104538,
 0.14: 0.01278867892920971,
 0.15: 0.014240797422826284,
 0.16: 0.015917663499712943,
 0.17: 0.017649030908942223,
 0.18: 0.01943587385118007,
 0.19: 0.021331265196204187,
 0.2: 0.023453339189291003,
 0.21: 0.025736050605773927,
 0.22: 0.028136154115200047,
 0.23: 0.03063242472708226,
 0.24: 0.0333886045217514,
 0.25: 0.036214739084243774,
 0.26: 0.03936804011464119,
 0.27: 0.042488856017589574,
 0.28: 0.04582953661680224,
 0.29: 0.049509342014789584,
 0.3: 0.053396073728799814,
 0.31: 0.05745161607861519,
 0.32: 0.06184772193431854,
 0.33: 0.0666947540640831,
 0.34: 0.07166790306568147,
 0.35: 0.076985427737236,


In [86]:
# discard top 20% of most extreme content
q_threshod = 1-0.2
p_threshold = q_preds[q_threshod]

#extremist_set = [d for d in predset if d['pred']>=p_threshold]
marginal_set = [d for d in predset if d['pred']>=q_preds[0.87] and d['pred']<q_preds[0.88]]
nonextremist_set = [d for d in predset if d['pred']>=q_preds[0.8] and d['pred']<q_preds[0.82]]


names_of_extremist_debates = sorted(list(set([
    d['OriginalDebateFileName'] for d in predset if d['pred']>=p_threshold
])))


In [90]:
names_of_extremist_debates[0:500]

['13th Aff - DDI 2020 AT.html5',
 '13th Amendment Case Neg - DDI 2020 GG.html5',
 '13th Amendment Neg - DDI 2020 FS.html5',
 '13th Neg - DDI 2020 AT.html5',
 '13th Neg - DDI 2020 KM.html5',
 '1ac Airports - DDI 2015 KQ.html5',
 '1ac Borders - DDI 2015 KQ.html5',
 '1ac critical financial surveillance pedagogy - DDI 2015 KS.html5',
 '2 for 1 DA - JDI 2017.html5',
 '2020 DA - Berkeley 2019.html5',
 '2020 DA - MSDI 2020.html5',
 '2020 Election DA - JDI 2020.html5',
 '2020 Election-Starter - Georgetown 2020.html5',
 '2020 Elections DA - Michigan7 2019 CCPW.html5',
 '2nd Session Packet - WTO topic - TDI 2021.html5',
 'A Door Into the Ocean Affirmative - HSS 2014.html5',
 'A Door Into the Ocean Negative - HSS 2014.html5',
 'AI Aff and Neg Updates - Northwestern 2018.html5',
 'AI Affirmative - Northwestern 2018.html5',
 'AIIB Aff Supplement - Michigan7 2016.html5',
 'AIIB Aff Wave 1 - Michigan7 2016.html5',
 'AIIB Aff-Neg - JDI 2016.html5',
 'AIIB Neg Starter - Michigan7 2016.html5',
 'AIIB Ne

In [91]:
names_of_extremist_debates[500:1000]

['Camp Tournament Updates - SDI 2018 DLGM.html5',
 'Camp Updates  - MichiganClassic 2017 OW.html5',
 'Camp Updates - Final - Michigan7 2017 AFMKK.html5',
 'Canada CP - Michigan7 2013 ACHM.html5',
 'Canada CP - Michigan7 2018 BFHPR.html5',
 'Cap Good Core - Wake 2018.html5',
 'Cap Good Updates - MichiganClassic 2021 MMP.html5',
 'Cap K  - JDI 2021.html5',
 'Cap K - Berkeley 2020 Wave 4.html5',
 'Cap K - DDIx 2021.html5',
 'Cap K - Links - Wake 2018.html5',
 'Cap K - Michigan7 2014.html5',
 'Cap K - SDI 2018 BGHT.html5',
 'Cap K - Starter Packet - Wake 2018.html5',
 'Cap K - UNT 2018.html5',
 'Cap K - Wake 2019.html5',
 'Cap K Links - Michigan7 2014.html5',
 'Cap K Preinstitute Set - Wake 2019.html5',
 'Cap K and Answers - Gonzaga 2020 LO.html5',
 'Cap K and Fem K Updates - SDI 2017.html5',
 'Cap K of Race based Affs - Northwestern 2014.html5',
 'Cap K vs Identity Teams - Northwestern 2015.html5',
 'Cap K vs K Affirmatives - Michigan7 2018 CPPWW.html5',
 'Cap K vs Race Affs - Michigan7 2

In [92]:
names_of_extremist_debates[1000:1500]

['Diplomatic Capital Disadvantage - Gonzaga 2013.html5',
 'Disability Aff - Berkeley 2017.html5',
 'Disability Aff - Wake 2017.html5',
 'Disability Aff Supplement - Berkeley 2017.html5',
 'Disability K - Michigan7 2018 FFGSV.html5',
 'Disability K - Wake 2017.html5',
 'Discourse Critique - Gonzaga 2014.html5',
 'Discourse Kritiks - DDI 2013 SS.html5',
 'Disruption Aff - Berkeley 2017.html5',
 'Disruption Neg - Berkeley 2017.html5',
 'Diversionary War Answers - Michigan7 2017.html5',
 'Diversity Visas Aff and Neg - Michigan7 2018 FFGSV.html5',
 'Do It Elsewhere CP - Northwestern 2014.html5',
 'Domestic Detention Affirmative - JDI 2015.html5',
 'Domestic Violence Aff - Starter Packet - Michigan7 2018.html5',
 'Domestic Violence Aff and Neg Supplement - MNDI 2018 GJJJ.html5',
 'Domestic Violence Aff and Neg and Gender K - Michigan7 2018 BFHPR.html5',
 'Domestic Violence Affirmative - MichiganClassic 2018.html5',
 'Domestic Violence Neg - Starter Packet - Michigan7 2018.html5',
 'Domestic 

In [93]:
names_of_extremist_debates[1500:2000]

['Impact Turns - UTNIF 2017.html5',
 'Impact Turns Aff   Neg - Michigan7 2019 BFHMRS.html5',
 'Impact Turns Core - Michigan7 2017 CBPPR.html5',
 'Impacts - China AI Leadership Good Bad - Michigan Classic 2022 HJV.html5',
 'Impacts - China Rise Good Bad - Michigan 7 2022 BFHR.html5',
 'Impacts - China War Good Bad - Michigan 7 2022 BEJJ.html5',
 'Impacts - Democracy Bad - Michigan 7 2022 BFHR.html5',
 'Impacts - Heg Bad - Michigan 7 2022 CPWW.html5',
 'Impacts - Heg Good - Michigan 7 2022 CPWW.html5',
 'Impacts - Heg Good Bad Supplement - Michigan 7 2022 BEJJ.html5',
 'Impacts - Hegemony - Mean Green 2022.html5',
 'Impacts - Impact Updates - Michigan Classic 2022 MMP.html5',
 'Impacts - LIO Good - Michigan 7 2022 BEJJ.html5',
 'Impacts - NATO Bad - Michigan 7 2022 BFHR.html5',
 'Impacts - NATO Good Bad - CNDI 2022.html5',
 'Impacts - NATO Good Bad - MSDI 2022.html5',
 'Impacts - NATO Good Bad - Michigan 7 2022 FMPS.html5',
 'Impacts - NATO Good Supplement - Michigan 7 2022 BEJJ.html5',


In [94]:
names_of_extremist_debates[2000:2500]

['NecroPolitics Aff  - Wake 2017.html5',
 'Necropolitics K - Wake 2017.html5',
 'Necropolitics Neg - Georgetown 2020.html5',
 'Neg - AI Clarity - Michigan Classic 2022 CGNO.html5',
 'Neg - AI Ethics Supplement - MNDI 2022 PHA.html5',
 'Neg - AI Interoperability - SDI 2022.html5',
 'Neg - AI LAWs - NAUDL 2022.html5',
 'Neg - AI Logistics - CNDI 2022.html5',
 'Neg - AI Logistics - Starter - Michigan 7 2022.html5',
 'Neg - AI Logistics Updates - Michigan 7 2022 FMPS.html5',
 'Neg - AI TEVV - MNDI 2022 PHA.html5',
 'Neg - Abolish ICE - MichiganClassic 2020 ACV.html5',
 'Neg - Ban OCOs - MSDI 2022.html5',
 'Neg - Cognitive Biotechnology - Michigan 7 2022 CPWW.html5',
 'Neg - Collateral Consequences - Michigan7 2020 BFHPR.html5',
 'Neg - Corporate Crime - Michigan7 2020 BFHPR.html5',
 'Neg - Cyber 5G - Michigan 7 2022 BEJJ.html5',
 'Neg - Cyber Info Sharing - Packet - SDI 2022.html5',
 'Neg - Cyber Space Assets - Michigan 7 2022 BEJJ.html5',
 'Neg - Cyber Space Assets - Michigan 7 2022 BFHR.

In [95]:
names_of_extremist_debates[2500:3000]

['SCS Grand Bargain Negative - MSDI 2016.html5',
 'SCS I-Law Neg - Michigan7 2016.html5',
 'SCS ILaw Aff - Michigan7 2016.html5',
 'SDWA Aff Neg - Michigan7 2021 BFPSW.html5',
 'SEC Affirmative - HSS 2015.html5',
 'SED Aff - Michigan7 2016.html5',
 'SED Neg - Michigan7 2016.html5',
 'SEL Neg - MSDI 2017.html5',
 'SIV Aff T Update - DDI 2018 AT.html5',
 'SIV Neg - DDI 2018 KM.html5',
 'SMR neg - DDI 2014 KQ.html5',
 'SP1 XO 12333 Negative - UTNIF 2015.html5',
 'SSD Aff and Neg 2.0 - Michigan7 2014 CHHJPV.html5',
 'SSD Affirmative - Michigan7 2014 CHHJPV.html5',
 'SSD Affirmative Upgrades - Michigan7 2014.html5',
 'SSD Negative Updates - Michigan7 2014.html5',
 'SSRA Aff Supplement - SDI 2015.html5',
 'SSRA Affirmative - Emory 2015.html5',
 'SSRA Affirmative - Northwestern 2015.html5',
 'SSRA Affirmative - SDI 2015.html5',
 'SSRA Affirmative and Negative - Northwestern 2015.html5',
 'STEM Aff  - Gonzaga 2017.html5',
 'STEM Aff  - MSDI 2017.html5',
 'STEM Aff - Starter Pack - UNT 2017.htm

In [96]:
names_of_extremist_debates[3000:]

['Vigilantism DA - MSDI 2020.html5',
 'Visa Aff - DDI 2017 AS.html5',
 'Visuality and Identity - DDI 2015 SWS.html5',
 'Vol CP + Ag DA - Gonzaga 2021.html5',
 'Vouchers Aff  - MSDI 2017.html5',
 'Vouchers Neg - MSDI 2017.html5',
 'WCC Neg - DDI 2020 FS.html5',
 'WOTUS Aff - DDI 2021 FJ.html5',
 'WOTUS Aff - DDIx 2021.html5',
 'WOTUS Aff - Michigan7 2021 EHJJPP.html5',
 'WOTUS Aff - SDI 2021.html5',
 'WOTUS Case Neg - DDI 2021 AT.html5',
 'WOTUS Case Neg - DDI 2021 KM.html5',
 'WOTUS Natives Aff Neg - SDI 2021.html5',
 'WOTUS Neg Supplement - MichiganClassic 2021 MMP.html5',
 'WOTUS Updates - Michigan7 2021 EHJJPP.html5',
 'Wag the Dog DA - Berkeley 2017.html5',
 'Wages DA - Berkeley 2018.html5',
 'Wages DA - Northwestern 2018.html5',
 'Wages DA - UNT 2018.html5',
 'Wages DA - Version 2 - SDI 2018 BJMSS.html5',
 'Wages DA - Wave 2 - Michigan7 2018 BFHPR.html5',
 'Wakanda CP - Michigan7 2019 K Lab.html5',
 'Wake Work Aff   Neg - Michigan7 2017 AFMMKK.html5',
 'War Impact Core - Michigan7

In [ ]:
# Crude keywords of possible extremism, to be validated by model after the fact to remove non-extremist content
possible_keywords_of_extremism = [s.lower() for s in [
    'white suprem', 'nazi','fascist','latcrit', 'critical consciousness', 'poststructuralist', 'capitalism','capitalist',
    'imperialist', 'imperialism', 'anti-colonial', 'Western man', 'Karl Marx',
    'heteronormative', 'hetero-normative', 'queer', 'LGBTQ', 'Marist', 'two-spirit',
    'Marxism', 'Trotsky', 'feminist', 'misogyny', 'Patriarchy', 'ecofeminist',
    'patriarchies', 'sexism', 'straight white male', 'white, heterosexual, and European men',
    'indigenous', 'indigenous', 'land back', 'decolonize', 'BIPOC', 'BLM', , 'eurocentric',
    'rapper', 'bitch', 'hip-hop', 'trans-movement', 'trans-identity', 'critical theory', 'praxis',
    'gender','transgender', 'trans-male', 'trans-female','critical', 'critique','Foucault','postmodern',
    'hegemony', 'hegemonic', 'gay','homosex','theory', 'revolution', 'movement', 'activist',
    'capitalist','capitalism','bourgeoisie','Bourgeois','sex','class consciousness',
    'lesbian','butch', 'post-modern','toxic masculin','thug',
    'black','people of color', 'reactionary', 'Trump', 'discourse','racial disparities',
    'anti-racist','antiracist','social justice','homophobic','xenophobic','white','black','latino','mexican','global south',
    'anti-poverty','criminal justice','progressive'
]]

def is_extremist_for_candidate_but_need_to_validate(x):
    """Crude filter of possible words that indicate extremism, to be validated by model after the fact to remove non-extremist content"""
    text = str(x['Full-Document']).lower()
    return any([kw in text for kew in possible_keywords_of_extremism])

SyntaxError: ignored